In [1]:
# ! unzip input/images/images.zip -d input/images/

In [2]:
# !pip install -q efficientnet_pytorch > /dev/null

In [3]:
import os
import torch
import torch.nn as nn
import helpers
import numpy as np 
import pandas as pd
from apex import amp
import torch.nn as nn
import pretrainedmodels
import albumentations as A
from sklearn import metrics
from torch.nn import functional as F
from models import SEResNext50_32x4d, SEResNeXt101_32x4d, NASnet
from Dataset import VehicleTrainDataset, VehicleTrainDataset2
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import random

warnings.filterwarnings("ignore", category=DeprecationWarning) 

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [4]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [5]:
# M = SEResNeXt101_32x4d(pretrained = 'imagenet')

In [6]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.1):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

In [7]:
def get_net():
    net = EfficientNet.from_pretrained('efficientnet-b7')
    net._fc = nn.Linear(in_features=2560, out_features=1, bias=True)
    return net

In [8]:
#Albumentations
def train_transforms():
    return A.Compose(
        [
            A.CenterCrop(height=224, width=224, p=0.2),
#             A.OneOf([
#                 A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
#                                      val_shift_limit=0.2, p=0.9),
#                 A.RandomBrightnessContrast(brightness_limit=0.2, 
#                                            contrast_limit=0.2, p=0.9),
#             ],p=0.9),
            A.ToGray(p=0.1),
#             A.CLAHE(p=5),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=5, p=0.9),
            A.Flip(p=0.5),
#             A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
#                                      val_shift_limit=0.2, p=0.5),
#             A.GaussianBlur(p=0.1),
#             A.GaussNoise(p=0.1),
#             A.OpticalDistortion(p=0.5),
            A.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225), max_pixel_value = 255., always_apply = True)
#             ToTensorV2(p=1.0),
        ]
    )


def valid_transforms():
    return A.Compose([
        A.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225), max_pixel_value = 255., always_apply = True)])

In [9]:
#Training functions
def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float()
def train_effdet(
    epoch,
    train_loader,
    model, 
    optimizer, 
    device,
    trainset, 
    validset,
    valid_loader,
    scheduler = None,
    iterations = 1,
    fp16 = False
):
    criterion = LabelSmoothing().to(device)
    
    if fp16:
        iterations = 1
    losses = helpers.AverageMeter()
    predictions = []
    model.train()
    if iterations > 1:
        optimizer.zero_grad()
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch, data in enumerate(tk0):
        for key, value in data.items():
            data[key] = value.to(device)
        if iterations == 1 and batch == 0:
            optimizer.zero_grad()
        
        targets = data["targets"].to(device).float()
        images = data["image"].to(device).float()
        
        out = model(images)
        loss = nn.BCEWithLogitsLoss()(out, targets.view(-1,1))
        
        with torch.set_grad_enabled(True):
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                
            else: 
                loss.backward()
            if (batch + 1) % iterations == 0:
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()
                if batch > 0:
                    optimizer.zero_grad()
        
        
        losses.update(loss.detach().item(), train_loader.batch_size)
        tk0.set_postfix(loss=losses.avg)
    
    
    ### validation
    val_losses = helpers.AverageMeter()
    final_predictions = []
    model.eval()
    with torch.no_grad():
        total_acc = 0.
        tk1 = tqdm(valid_loader, total=len(valid_loader))
        for batch, data in enumerate(tk1):
            for key , value in data.items():
                data[key] = value.to(device)
            targets = data["targets"].to(device).float()
            images = data["image"].to(device).float()

            out = model(images)
            loss = nn.BCEWithLogitsLoss()(out, targets.view(-1,1))
            predictions = out.cpu()
            val_losses.update(loss.detach().item(), valid_loader.batch_size)
#             predictions = nn.functional.softmax(predictions, dim=1).data.cpu().numpy()[:,1]
            final_predictions.append(predictions.view(-1,1))
            tk1.set_postfix(loss=val_losses.avg)
    
    return final_predictions, losses.avg, val_losses.avg

In [10]:
def train(
    epoch,
    train_loader,
    model, 
    optimizer, 
    device,
    trainset, 
    validset,
    valid_loader,
    scheduler = None,
    iterations = 1,
    fp16 = False
):    
    if fp16:
        iterations = 1
    losses = helpers.AverageMeter()
    predictions = []
    model.train()
    if iterations > 1:
        optimizer.zero_grad()
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch, data in enumerate(tk0):
        for key, value in data.items():
            data[key] = value.to(device)
        if iterations == 1 and batch == 0:
            optimizer.zero_grad()
        
        out, loss = model(**data)
        
        with torch.set_grad_enabled(True):
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                
            else: 
                loss.backward()
            if (batch + 1) % iterations == 0:
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()
                if batch > 0:
                    optimizer.zero_grad()
        
        
        losses.update(loss.item(), train_loader.batch_size)
        tk0.set_postfix(loss=losses.avg)
    
    
    ### validation
    val_losses = helpers.AverageMeter()
    final_predictions = []
    model.eval()
    with torch.no_grad():
        tk1 = tqdm(valid_loader, total=len(valid_loader))
        for batch, data in enumerate(tk1):
            for key , value in data.items():
                data[key] = value.to(device)
            out, val_loss = model(**data)
            predictions = out.cpu()
            val_losses.update(loss.detach().item(), valid_loader.batch_size)
            final_predictions.append(predictions)
#             total_acc += accuracy(out, data["targets"].view(-1,1))
            tk1.set_postfix(loss=val_losses.avg)
    nv = len(valid_loader)
    
    return final_predictions, losses.avg, val_losses.avg#total_acc/nv).cpu().numpy()

In [11]:
def main(fold = None):
    print(f"[FOLD]: {fold}")
    train_data = "input/images/images"
    df = pd.read_csv("input/train_folds.csv")
    train_bs = 32
    valid_bs = 16

    # kfold validation
    train_df = df[df.kfold != fold].reset_index(drop = True)
    valid_df = df[df.kfold.isin([fold])].reset_index(drop = True)

    trainset = VehicleTrainDataset(train_df, image_dir = train_data,
                                         transforms = train_transforms())
    validset = VehicleTrainDataset(valid_df, image_dir = train_data,
                                         transforms = valid_transforms())
    
    #### effdet
#     trainset = VehicleTrainDataset2(train_df, train_df.emergency_or_not.values, image_dir = train_data,
#                                          transforms = train_transforms())
#     validset = VehicleTrainDataset2(valid_df, valid_df.emergency_or_not.values, image_dir = train_data,
#                                          transforms = valid_transforms())

#     print(len(trainset))
#     print(len(validset))

    trainloader = DataLoader(
        dataset = trainset,
        batch_size=train_bs,
        shuffle=True,
        num_workers=4
    )
    
    validloader = DataLoader(
        dataset = validset,
        batch_size=valid_bs,
        shuffle=False,
        num_workers=4
    )
    
    device = "cuda"
    epochs = 50
    lr = 1e-4
#     lr = 0.0005
    
#     model = NASnet(pretrained = "imagenet+background")
    model = SEResNeXt101_32x4d(pretrained = 'imagenet')
#     model = get_net().cuda()
    model_path = f"Output/{type(model).__name__}_{fold}.pth"
    model.to(device)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    
    optimizer = torch.optim.AdamW(optimizer_parameters, lr = lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.8,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    
#     amp.register_float_function(torch, 'sigmoid')
    
    model, optimizer = amp.initialize(
        model, 
        optimizer,
        opt_level="O1",
        verbosity=0
    )
    
    es = helpers.EarlyStopping(patience = 5, mode="max")
    
    for epoch in range(epochs):
        predictions, train_loss, valid_loss = train(
            epoch,
            trainloader,
            model, 
            optimizer, 
            device,
            trainset, 
            validset,
            validloader,
            iterations = 1,
            fp16 = True)
        
        predictions = np.vstack((predictions)).ravel()
#         targets = np.array(valid_df.emergency_or_not) >= 0.5
        predictions = [0 if i < 0.5 else 1 for i in predictions]
        acc = metrics.accuracy_score(valid_df.emergency_or_not, predictions)
        
#         acc = metrics.roc_auc_score(targets, predictions)
        scheduler.step(acc)
        
        print(f"Epoch: {epoch} train_loss:{round(train_loss,3)}, valid_loss: {round(valid_loss,3)} acc: {acc:.3f}")
        es(acc, model, model_path)
        if es.early_stop:
            print("Early Stopping!")
            break

In [ ]:
main(0)
main(1)
main(2)
# main(3)
# main(4)

[FOLD]: 0


100%|██████████| 21/21 [00:03<00:00,  6.37it/s, loss=0.0363]


Epoch: 0 train_loss:0.288, valid_loss: 0.036 acc: 0.924
Validation score improved (-inf --> 0.9242424242424242). Saving model!


100%|██████████| 21/21 [00:01<00:00, 11.20it/s, loss=0.127]


Epoch: 1 train_loss:0.116, valid_loss: 0.127 acc: 0.945
Validation score improved (0.9242424242424242 --> 0.9454545454545454). Saving model!


...

Epoch: 4 train_loss:0.041, valid_loss: 0.222 acc: 0.948
EarlyStopping counter: 1 out of 5


  0%|          | 0/42 [00:00<?, ?it/s]

Epoch: 5 train_loss:0.048, valid_loss: 0.538 acc: 0.945
EarlyStopping counter: 2 out of 5


  0%|          | 0/42 [00:00<?, ?it/s]

Epoch: 6 train_loss:0.064, valid_loss: 0.357 acc: 0.948
EarlyStopping counter: 3 out of 5
